In [2]:
import pandas as pd
import numpy as np

import json
import csv
import matplotlib.pyplot as plt
import collections
import string
import math
import time

from itertools import groupby
from pprint import pprint as pp
from pyjarowinkler import distance

In [3]:
# available char 는 ASCII 코드 33 ~ 126 까지의 값들의 모임이다.

available_char = string.digits+string.ascii_letters+string.punctuation
print(available_char)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [4]:
def Available(string):
    for element in string:
        if element not in available_char:
            return np.nan
    else:
        return string

In [ ]:
colnames = ['ID','Domain', 'PW']


for char in string.ascii_letters:
    
    
    path = './3Billion/' +char + '-sorted-uniq.csv'
    print(path)
    
    data = pd.read_csv(path, names = colnames, header=None, error_bad_lines=False)
        
    print(len(data), '<- data의 원래 길이:' )
    
    data = data.dropna(axis=0)
    print(len(data), 'NaN제거후의 길이')
    
    data['PW'] = data['PW'].apply(Available)
    data = data.dropna(how='any')
    print(len(data),'<- PW 칼럼에서 noise제거후 길이' )

    data['ID'] = data['ID'].apply(Available)
    data = data.dropna(how='any')
    print(len(data),'<- ID 칼럼에서 noise제거후 길이' )

    data['Domain'] = data['Domain'].apply(Available)
    data = data.dropna(how='any')
    print(len(data),'<- Domain 칼럼에서 noise제거후 길이')
    
    
    save_path = './Noise_Clean_3Billion/'+char+'.pickle'
    data.to_pickle(save_path)
        
    print('')


In [ ]:
colnames = ['ID','Domain', 'PW']
path = './3Billion/numbers/num.csv'

data = pd.read_csv(path, names = colnames, header=None, error_bad_lines=False)
print(len(data), '<- data의 원래 길이:' )

data = tmp.dropna(axis=0)
print(len(data),'<-  NaN제거후의 길이' )

data['PW'] = data['PW'].apply(Available)
data = data.dropna(how='any')
print(len(data),'<- PW 칼럼에서 noise제거후 길이' )

data['ID'] = data['ID'].apply(Available)
data = data.dropna(how='any')
print(len(data),'<- ID 칼럼에서 noise제거후 길이' )

data['Domain'] = data['Domain'].apply(Available)
data = data.dropna(how='any')
print(len(data),'<- Domain 칼럼에서 noise제거후 길이')

data = data.reset_index(drop=True)

# save
data.to_pickle('./Noise_Clean_3Billion/num.pickle')

In [ ]:
for char in string.ascii_letters:
    
    
    path = './Noise_Clean_3Billion/' +char + '.pickle'
    #print(path)
    data = pd.read_pickle(path)
    name = char +'.pickle:'
    print(name, len(data))
    print('')

In [ ]:
path = './num' + '.pickle'
data = pd.read_pickle(path)
name = 'num' +'.pickle:'
print(name, len(data))

In [ ]:
import re
import time
import string
from datasketch import MinHash, MinHashLSHForest

import os
import pickle
import random
import pandas as pd
from pyjarowinkler import distance

In [ ]:
def preprocess(text):
    num_gram = 4
    tokens = [text[i:i+num_gram] for i in range(len(text)-num_gram+1)]

    return tokens

def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['ID']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

#def predict(text, database, perms, num_results, forest):
#def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    #result = database.iloc[idx_array]['id']
    #result_pw = database.iloc[idx_array]['password']
    result = idx_array
    
    #print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

def predict(text, perms, num_results, forest):

    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = idx_array
    
    return result

def hash_list(values):
    total = '0'
    for i in range(len(values)):
        total += values[i]    
    result = hash(total)
    #print(total)
    
    return(result)

def lsh(num):
    
    a= data['ID'][num]
    b = data['PW'][num]
    refer = [a,b]
    
    idx = num
    compare_id = refer[0]
    
    predict_result = predict(compare_id, permutations, num_recommendations, forest)
    
    #lsh_data = data.iloc[sorted(predict_result)]    
    return(predict_result)

def Hashing(hash_list):
    total = '0'
    for i in range(len(hash_list)):
        total += hash_list[i]    
    result = hash(total)
    #print(total)
    
    return(result)

def PWset(idx):
    reference = data['PW'][idx]
    
    result  = lsh(idx)
    temp_df = data.loc[sorted(result)]
    
    PW = list(temp_df['PW'].values)
    PWset = list()
    
    for password in PW:
        temp = distance.get_jaro_distance(reference, password)
        #print(reference, password)
        if 1>= temp > threshold:
            PWset.append(password)
    PWset = list(set(PWset))
    
    if len(PWset)>1:
        return PWset

In [ ]:
def SaveForest(forest, seq):
    
    save_path = './LSH_Pickle/'+'s/'+'forest_s_'+str(seq) +'.pickle'
    with open (save_path, 'wb') as f:
        pickle.dump(forest,f)
    print(seq, 'forest_saved')
    
def SavePWset(data, seq):
        
    data = data.dropna(how='any')
    data['HashTag'] = data['PWset'].apply(Hashing)
    data = data.drop_duplicates(subset='HashTag', keep ='first')
    
    
    save_path = './LSH_Data/'+'s/'+'s_'+str(seq) +'.pickle'
    data.to_pickle(save_path)
    print(seq,'Data Pickle Saved')
    print('')

In [ ]:
#Number of Permutations

threshold = 0.8
permutations = 32
num_recommendations = 30

In [ ]:
for i in range(25):
    
    path = './For_LSH_data/s/'+'s_'+str(i)+'.pickle'
    data = pd.read_pickle(path)
    print(data.head())
    
    forest = get_forest(data, permutations)
    
    SaveForest(forest,i)
    
    data = data.assign(index=lambda x: data.index)
    start = time.time()
    data['PWset'] = data['index'].apply(PWset)
    current = time.time() - start
    print('Making PWset consumed time:' ,current)
    
    SavePWset(data, i)
    